# Medication and diagnosis visualization

### Aggregating by country and in general

As part of the phase 1.1 we have explored the relation between medication and diagnosis representing in a scatter plot: % patients with a diagnostic vs % patients on a mediacation. The plots are focused only on values before admission, and each scatter plot includes two data series: one for never severe and another one for ever severe. Each dot represent a site and the value for never severe, while each triangle represent a site and the value for ever severe. We aggregated the results by: country and in general.

In [ ]:
#############
# LIBRARIES #
#############
library(dplyr)
library(tidyr)
library(ggplot2)

Set the working directory where the files are (using as input the files in "latest")

In [ ]:
setwd("./4CE/phase1.1/")

In [ ]:
#function to create a list with the files to analyze
fileList <- function( path, pattern, pediatric ){
  
  fileListInput <- list.files( path = path,pattern = pattern)
  #remove FICHOS
  fileListInput  <- fileListInput[! grepl( paste( "FICHOS" , collapse = "|"), x = fileListInput)]
  
  if( pediatric == TRUE){
    fileListInput  <- fileListInput[ grepl( paste( "PED", collapse = "|"), x = fileListInput)]
  }else{
    fileListInput  <- fileListInput[ !grepl( paste( c("PED", "BCH", "CHOP", "RP401" ), collapse = "|"), x = fileListInput)]
  }
  return( fileListInput)
}

Create a list with all the files distinguishing between:
- Demographics data
- Medication data
- Diagnosis data

We will distinguish between pediatric and non pediatric (select_pediatric TRUE or FALSE)
The list of the current sites with pediatric data are detailed in the vector 'pediatricSites'

In [ ]:
select_pediatric = FALSE
fileListDiag <- fileList( path = "./latest/",pattern = "Diag", pediatric = select_pediatric)
fileListMed <- fileList( path = "./latest/",pattern = "Med", pediatric = select_pediatric)
fileListDemog <- fileList( path = "./latest/",pattern = "Demog", pediatric = select_pediatric)

Put together all the diagnosis data from the distinct sites

In [ ]:
for( i in 1:length( fileListDiag ) ){
  print(i)
  selection <- read.delim( paste0( "./latest/",fileListDiag[i]), sep = ",", colClasses = "character")
  colnames(selection) <- tolower( colnames( selection ) )
  
  if( i== 1){
    allDiagnosis <- selection
  }else{ 
    allDiagnosis <- rbind( allDiagnosis, selection )
  }
}
#transform column names to know they are referring to diagnosis
colnames( allDiagnosis)[4:7] <- paste0( colnames( allDiagnosis)[4:7], "_diagnosis")

Put together all the medication data from the distinct sites

In [ ]:
for( i in 1:length( fileListMed ) ){
  print(i)
  selection <- read.delim( paste0( "./latest/",fileListMed[i]), sep = ",", colClasses = "character")
  colnames(selection) <- tolower( colnames( selection ) )
  
  if( i== 1){
    allMedication <- selection
  }else{ 
    allMedication <- rbind( allMedication, selection )
  }
}
#transform column names to know they are referring to medication
colnames( allMedication)[3:6] <- paste0( colnames( allMedication)[3:6], "_medication")

Put together all the demographic data from the distinct sites

In [ ]:
for( i in 1:length( fileListDemog ) ){
  print(i)
  selection <- read.delim( paste0( "./latest/",fileListDemog[i]), sep = ",", colClasses = "character")
  colnames( selection ) <- tolower( colnames( selection ))
  selection <- selection[ selection$sex == "all" &
                            selection$age_group == "all" & 
                            selection$race == "all", ]
  
  if( i== 1){
    allDemographics <- selection
  }else{ 
    allDemographics <- rbind( allDemographics, selection )
  }
}

#estimate the total never severe per site and change the column names to know they are refered to total counts per site
allDemographics[ allDemographics < 0 ] <- NA
allDemographics$num_patients_never_severe <- as.numeric(allDemographics$num_patients_all) - as.numeric(allDemographics$num_patients_ever_severe)
allDemographics<- allDemographics[, c("siteid", "num_patients_all", "num_patients_ever_severe", "num_patients_never_severe")]
colnames(allDemographics) <- c("siteid", "totalPatients", "totalEverSever", "totalNeverSevere")

Put all the information together mergin all dataframes by siteid

In [ ]:
finalDataSet <- merge( allDiagnosis, allMedication, by = "siteid")
finalDataSet <- merge( finalDataSet, allDemographics, by = "siteid")

Transform obfuscated and unknown values (-99, -999) into NA

In [ ]:
finalDataSet[,c(4:7,9:15)] <- sapply(finalDataSet[,c(4:7,9:15)],as.numeric)
finalDataSet[ finalDataSet < 0 ] <- NA

Group the data by country or by all. To aggregate and avoid duplications (same description for ICD9 and ICD10 code), we aggregate by the description of the ICD code. 

In [ ]:
groupData <- function( input, aggregation ){
  if( aggregation == "Country"){
    siteMaping <- read.delim("./mappingFiles/SiteID_Map.csv", colClasses = "character", sep=",")
    siteMaping <- siteMaping[,c("Acronym", "Country", "Country.Color")]
    toanalyze <- merge( input, siteMaping, by.x = "siteid", by.y = "Acronym")
    
    toanalyzebyCountry <- toanalyze[, -1]
    
    #icd description
    mapping <- read.delim("./mappingFiles/2020AA_Icd9_Icd10_Dictionary.txt", colClasses = "character")
    colnames(mapping) <- c("icd", "icd_version", "description" )
    #we map according to the pair ICD + version to avoid issues when same code is present in ICD9 and ICD10 but refer to different diagnosis
    mapping$icdPair <- paste0( mapping$icd, "-", mapping$icd_version)
    
    toanalyzebyCountry$icdPair <- paste0( toanalyzebyCountry$icd_code_3chars, "-", toanalyzebyCountry$icd_version)
    toanalyzebyCountry <- merge( toanalyzebyCountry, mapping, by = "icdPair")
    toanalyzebyCountry <- toanalyzebyCountry[, c(4:17,20)]
    
      byCountryOutput <- toanalyzebyCountry %>% 
      group_by(Country, Country.Color, description, med_class) %>% 
      dplyr::summarise_all(~{sum(.x, na.rm = any(!is.na(.x)))})
    
    output <- as.data.frame( byCountryOutput )
  }
  if( aggregation == "All"){
    byAll <- input[, -1]
    
    #icd description
    mapping <- read.delim("./mappingFiles/2020AA_Icd9_Icd10_Dictionary.txt", colClasses = "character")
    colnames(mapping) <- c("icd", "icd_version", "description" )
    #we map according to the pair ICD + version to avoid issues when same code is present in ICD9 and ICD10 but refer to different diagnosis
    mapping$icdPair <- paste0( mapping$icd, "-", mapping$icd_version)
    
    byAll$icdPair <- paste0( byAll$icd_code_3chars, "-", byAll$icd_version)
    byAll <- merge( byAll, mapping, by = "icdPair")
    byAll <- byAll[, c(4:15,18)]
    
    byAllOutput <- byAll %>% 
      group_by(description, med_class) %>% 
      dplyr::summarise_all(~{sum(.x, na.rm = any(!is.na(.x)))})
    output <- as.data.frame( byAllOutput )
  }
  return( output )
}


#aggregationType <- "All"
aggregationType <- "Country"
toanalyze <- groupData( input = finalDataSet, aggregation = aggregationType )

Add percentage estimations:
- ever severe and on X medication / total ever severe
- ever severe and with X disease / total ever severe
- never severe qnd on X medication / total never severe
- never severe and with X disease / total never severe

In [ ]:
# ever severe and on X medication / total ever severe
# ever severe and with X disease / total ever severe
toanalyze$ratio_everSevereOnMedication_beforeAdmission <-  round( 100*(toanalyze$num_patients_ever_severe_before_admission_medication / toanalyze$totalEverSever), 3)
toanalyze$ratio_everSevereWithDiagnosis_beforeAdmission <- round( 100*(toanalyze$num_patients_ever_severe_before_admission_diagnosis / toanalyze$totalEverSever), 3)

# never severe qnd on X medication / total never severe
# never severe and with X disease / total never severe
toanalyze$num_patients_never_severe_before_admission_diagnosis <- toanalyze$num_patients_all_before_admission_diagnosis - toanalyze$num_patients_ever_severe_before_admission_diagnosis
toanalyze$num_patients_never_severe_before_admission_medication <- toanalyze$num_patients_all_before_admission_medication - toanalyze$num_patients_ever_severe_before_admission_medication
toanalyze$ratio_nonSevereOnMedication_beforeAdmission <-  round( 100*( toanalyze$num_patients_never_severe_before_admission_medication / toanalyze$totalNeverSevere), 3)
toanalyze$ratio_nonSevereWithDiagnosis_beforeAdmission <- round( 100*( toanalyze$num_patients_never_severe_before_admission_diagnosis/ toanalyze$totalNeverSevere), 3)

Remove negative values and values > 100 that can appear due to obfucation blurring

In [ ]:
colnum <- which(colnames( toanalyze) %in% c("ratio_everSevereOnMedication_beforeAdmission",
                                            "ratio_everSevereWithDiagnosis_beforeAdmission",
                                            "ratio_nonSevereOnMedication_beforeAdmission",
                                            "ratio_nonSevereWithDiagnosis_beforeAdmission"))
toanalyze[ , c(colnum) ][ toanalyze[ , c(colnum) ] > 100 ] <- NA

Transform the format of data.frame to plot ever and never severe in the same plot

In [ ]:
if( aggregationType == "Country"){
  everSevere <- toanalyze[ , c("Country", "med_class", "description", "ratio_everSevereOnMedication_beforeAdmission", "ratio_everSevereWithDiagnosis_beforeAdmission", "Country.Color")]
  everSevere$status <- "Severe"
  colnames(everSevere) <- gsub("ratio_everSevere", "",colnames( everSevere))
  
  neverSevere <- toanalyze[ , c("Country", "med_class", "description", "ratio_nonSevereOnMedication_beforeAdmission", "ratio_nonSevereWithDiagnosis_beforeAdmission", "Country.Color")]
  neverSevere$status <- "Never Severe"
  colnames(neverSevere) <- gsub("ratio_nonSevere", "",colnames( neverSevere))
  
  toplot <- rbind( everSevere, neverSevere)
  save( toplot, file="./toplotByCountry.RData")
  
}

if( aggregationType == "All"){
  everSevere <- toanalyze[ , c("med_class", "description", "ratio_everSevereOnMedication_beforeAdmission", "ratio_everSevereWithDiagnosis_beforeAdmission")]
  everSevere$status <- "Severe"
  colnames(everSevere) <- gsub("ratio_everSevere", "",colnames( everSevere))
  
  neverSevere <- toanalyze[ , c("med_class", "description", "ratio_nonSevereOnMedication_beforeAdmission", "ratio_nonSevereWithDiagnosis_beforeAdmission")]
  neverSevere$status <- "Never Severe"
  colnames(neverSevere) <- gsub("ratio_nonSevere", "",colnames( neverSevere))
  
  toplot <- rbind( everSevere, neverSevere)
  save( toplot, file="./toplotAll.RData")
  
}

Example of the scatter plot generated. 

In [ ]:
medication_class <- "COAGB"
diagnosis_group <- "Acute kidney failure"

if( aggregationType == "All"){
  datatoplot <- toplot[ toplot$description == diagnosis_group &
                          toplot$med_class == medication_class,]
  datatoplot <- datatoplot[ complete.cases( datatoplot ), ]
  
  xlim <- max( c( datatoplot$WithDiagnosis_beforeAdmission), na.rm = TRUE)
  ylim <- max( c( datatoplot$OnMedication_beforeAdmission), na.rm = TRUE)
  
  
  ggplot( data = datatoplot, mapping = aes( x = WithDiagnosis_beforeAdmission, 
                                            y = OnMedication_beforeAdmission), 
          shape = status) +
    geom_point( aes(shape= factor( status)), size = 2, 
                color = "grey")+
    theme(legend.position = "bottom", 
          plot.title = element_text(hjust = 0.5, size = 10), 
          axis.title.x = element_text(size = 8), 
          axis.title.y = element_text(size = 8)) +
    scale_x_continuous(name= paste0( "% patients with ", medication_class," medication class"), limits=c(0, xlim+1)) +
    scale_y_continuous(name= paste0( "% patients with ", diagnosis_group), limits=c(0, ylim+1)) +
    labs(title= paste0( medication_class, " - ", diagnosis_group ), shape = "Severity status")
  
}

if( aggregationType == "Country"){
  datatoplot <- toplot[ toplot$description == diagnosis_group &
                          toplot$med_class == medication_class,]
  datatoplot <- datatoplot[ complete.cases( datatoplot ), ]
  
  xlim <- max( c( datatoplot$WithDiagnosis_beforeAdmission), na.rm = TRUE)
  ylim <- max( c( datatoplot$OnMedication_beforeAdmission), na.rm = TRUE)
  
  
  ggplot( data = datatoplot, mapping = aes( x = WithDiagnosis_beforeAdmission, 
                                            y = OnMedication_beforeAdmission), 
          shape = status) +
    geom_point( aes(shape= factor( status)), size = 2, 
                alpha = 0.5, 
                color = datatoplot$Country.Color)+
    geom_text(aes(label=Country),hjust=0, vjust=0, size = 2)+
    theme(legend.position = "bottom", 
          plot.title = element_text(hjust = 0.5, size = 10), 
          axis.title.x = element_text(size = 8), 
          axis.title.y = element_text(size = 8)) +
    scale_x_continuous(name= paste0( "% patients with ", medication_class," medication class"), limits=c(0, xlim+1)) +
    scale_y_continuous(name= paste0( "% patients with ", diagnosis_group), limits=c(0, ylim+1)) +
    labs(title= paste0( medication_class, " - ", diagnosis_group ), shape = "Severity status")
  
}